**산탄데르 고객 만족 예측**

In [ ]:
%cd '/content/drive/MyDrive/[EURON] 파이썬 머신러닝 완벽 가이드'

/content/drive/MyDrive/[EURON] 파이썬 머신러닝 완벽 가이드


In [ ]:
# 라이브러리 임포트, 데이터 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')

cust_df = pd.read_csv('cust_train.csv',encoding='latin-1')
print(cust_df.shape)
cust_df.head()

(76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [ ]:
# 피처 타입 확인
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [ ]:
# 클래스 분포 확인 -> 불균형한 데이터
print(cust_df['TARGET'].value_counts())
unsatisfied_cnt = cust_df[cust_df['TARGET']==1].TARGET.count()
total_cnt = cust_df.TARGET.count()
print('unsatisfied 비율 : {0:.3f}'.format(unsatisfied_cnt/total_cnt))

0    73012
1     3008
Name: TARGET, dtype: int64
unsatisfied 비율 : 0.040


In [ ]:
# 데이터 전처리
# var3 피처값 2로 대체, ID 피처 드롭
cust_df['var3'].replace(-999999,2,inplace=True)
cust_df.drop('ID',axis=1, inplace=True)

# 피처셋과 레이블셋 분리
X_features = cust_df.iloc[:,:-1]
y_labels = cust_df.iloc[:,-1]
print('피처 데이터 shape : {0}'.format(X_features.shape))

피처 데이터 shape : (76020, 369)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size = 0.2)

In [ ]:
# XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimators = 100, random_state = 111)
xgb_clf.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc", eval_set=[(X_train,y_train),(X_test,y_test)])
xgb_roc = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1], average='macro')
print('xgb ROC AUC : {0:.4f}'.format(xgb_roc))

[0]	validation_0-auc:0.80166	validation_1-auc:0.794572
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.808168	validation_1-auc:0.80141
[2]	validation_0-auc:0.813133	validation_1-auc:0.805834
[3]	validation_0-auc:0.820194	validation_1-auc:0.81156
[4]	validation_0-auc:0.818855	validation_1-auc:0.81115
[5]	validation_0-auc:0.819913	validation_1-auc:0.812425
[6]	validation_0-auc:0.822231	validation_1-auc:0.814681
[7]	validation_0-auc:0.822284	validation_1-auc:0.814655
[8]	validation_0-auc:0.820481	validation_1-auc:0.813158
[9]	validation_0-auc:0.822686	validation_1-auc:0.814462
[10]	validation_0-auc:0.82111	validation_1-auc:0.812574
[11]	validation_0-auc:0.823725	validation_1-auc:0.815358
[12]	validation_0-auc:0.82157	validation_1-auc:0.813763
[13]	validation_0-auc:0.824547	validation_1-auc:0.816851
[14]	validation_0-auc:0.823239	validation_1-auc:0.815709
[15]

In [ ]:
# XGB 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[5,7], 'min_child_weight':[1,3],'colsample_bytree':[0.5,0.75]}

gridcv = GridSearchCV(xgb_clf, param_grid=params, cv=3)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc", eval_set=[(X_train,y_train),(X_test,y_test)])
print('GridCV 최적 파라미터 : ', gridcv.best_params_)

xgb_roc = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('xgb 튜닝 ROC AUC : {0:.4f}'.format(xgb_roc))

[0]	validation_0-auc:0.728853	validation_1-auc:0.696515
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.733976	validation_1-auc:0.701166
[2]	validation_0-auc:0.806579	validation_1-auc:0.787294
[3]	validation_0-auc:0.819201	validation_1-auc:0.805313
[4]	validation_0-auc:0.81681	validation_1-auc:0.801082
[5]	validation_0-auc:0.8156	validation_1-auc:0.798983
[6]	validation_0-auc:0.816656	validation_1-auc:0.799198
[7]	validation_0-auc:0.818773	validation_1-auc:0.801316
[8]	validation_0-auc:0.823592	validation_1-auc:0.808642
[9]	validation_0-auc:0.825092	validation_1-auc:0.810685
[10]	validation_0-auc:0.824595	validation_1-auc:0.80849
[11]	validation_0-auc:0.829133	validation_1-auc:0.81372
[12]	validation_0-auc:0.827253	validation_1-auc:0.811438
[13]	validation_0-auc:0.827551	validation_1-auc:0.810966
[14]	validation_0-auc:0.825995	validation_1-auc:0.808366
[15

In [ ]:
# LightGBM
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(n_estimators = 100, random_state = 111)
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc", eval_set=[(X_test,y_test)], verbose=True)
lgbm_roc = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1], average='macro')
print('lgbm ROC AUC : {0:.4f}'.format(lgbm_roc))

[1]	valid_0's binary_logloss: 0.157276	valid_0's auc: 0.816388
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.152734	valid_0's auc: 0.821579
[3]	valid_0's binary_logloss: 0.149346	valid_0's auc: 0.82612
[4]	valid_0's binary_logloss: 0.146755	valid_0's auc: 0.829159
[5]	valid_0's binary_logloss: 0.144686	valid_0's auc: 0.830499
[6]	valid_0's binary_logloss: 0.142975	valid_0's auc: 0.832772
[7]	valid_0's binary_logloss: 0.141492	valid_0's auc: 0.836947
[8]	valid_0's binary_logloss: 0.140343	valid_0's auc: 0.836478
[9]	valid_0's binary_logloss: 0.139318	valid_0's auc: 0.837447
[10]	valid_0's binary_logloss: 0.13843	valid_0's auc: 0.838018
[11]	valid_0's binary_logloss: 0.137701	valid_0's auc: 0.838661
[12]	valid_0's binary_logloss: 0.137064	valid_0's auc: 0.83886
[13]	valid_0's binary_logloss: 0.136479	valid_0's auc: 0.839669
[14]	valid_0's binary_logloss: 0.136003	valid_0's auc: 0.839809
[15]	valid_0's binary_logloss: 0.135713	valid_0's auc:

In [ ]:
# LightGBM 하이퍼 파라미터 튜닝
params = {'num_leaves':[32,64],'max_depth':[128,160], 'min_child_samples':[60,100], 'subsample':[0.8,1]}

gridcv = GridSearchCV(lgbm_clf, param_grid=params, cv=3)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc", eval_set=[(X_train,y_train),(X_test,y_test)])
print('GridCV 최적 파라미터 : ', gridcv.best_params_)

lgbm_roc = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('lgbm 튜닝 ROC AUC : {0:.4f}'.format(lgbm_roc))

[1]	valid_0's binary_logloss: 0.158083	valid_0's auc: 0.824753	valid_1's binary_logloss: 0.157375	valid_1's auc: 0.816055
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.152738	valid_0's auc: 0.831073	valid_1's binary_logloss: 0.152978	valid_1's auc: 0.81862
[3]	valid_0's binary_logloss: 0.149011	valid_0's auc: 0.835618	valid_1's binary_logloss: 0.149772	valid_1's auc: 0.821814
[4]	valid_0's binary_logloss: 0.146014	valid_0's auc: 0.838424	valid_1's binary_logloss: 0.147089	valid_1's auc: 0.825358
[5]	valid_0's binary_logloss: 0.143572	valid_0's auc: 0.842748	valid_1's binary_logloss: 0.145006	valid_1's auc: 0.830011
[6]	valid_0's binary_logloss: 0.141615	valid_0's auc: 0.845221	valid_1's binary_logloss: 0.143373	valid_1's auc: 0.831241
[7]	valid_0's binary_logloss: 0.140006	valid_0's auc: 0.846157	valid_1's binary_logloss: 0.142047	valid_1's auc: 0.831389
[8]	valid_0's binary_logloss: 0.138547	valid_0's auc: 0.849035	valid_1's binary_loglo

In [ ]:
# 피처별 중요도
from xgboost import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(1,1,figsize=(10,8))
plot_importance(xgb_clf, ax=ax, max_num_features=20, height=0.4)